In [1]:
from math import pi 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [2]:
# Construct background mesh
Omega = Rectangle(1, 1).Face() 
Omega.faces.name = "Omega"
Omega.edges.Min(X).name = "left"
Omega.edges.Min(Y).name = "bottom"
Omega.edges.Max(X).name = "right"
Omega.edges.Max(Y).name = "top"
mesh = Mesh(OCCGeometry(Omega, dim=2).GenerateMesh(maxh=0.01))
# print(mesh.GetBoundaries())

In [3]:
fp = 0
ff = 0
xi = 0.55
Kp = 1
d = 0.01

In [4]:
# case = "Permeable"
case = "Impermeable"
if case == "Permeable":
    dirichlet_bnd = 'left|right|top|bottom'
    neumann_bnd = ''
    dirichlet_bndf = ".*"
    Kft = IfPos(y - 0.5, 1e6, Kp)
    Kfn = IfPos(y - 0.5, 1e2, Kp)
    epi = 1e-7
    gD = IfPos((y-epi)*(y-1+epi), 0, x+1)
    gN = 0
elif case =="Impermeable":
    dirichlet_bnd = 'left|right'
    neumann_bnd = 'top|bottom'
    dirichlet_bndf = ""
    Kft = IfPos(y - 0.5, 1e-7, Kp)
    Kfn = IfPos(y - 0.5, 1e-7, Kp)
    gD = x
    gN = 0


In [5]:
alphaf = 4*Kfn/(2*xi-1)/d
betaf = Kfn/d
beta_p = 200 # penalization parameter
sigma_p = 0.01
sigma_f = 0.01

In [6]:
# Define the levelset function for the interface
levelset = x-1/2
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
# DrawDC(lsetp1,-1,1,mesh,'lsetp1')

ci = CutInfo(mesh,lsetp1)
haspos = ci.GetElementsOfType(HASPOS)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(haspos),mesh)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)
# Draw(BitArrayCF(ci.GetElementsOfType(NEG)),mesh)

In [7]:
# Construct the unfitted fem space 
Phbase = L2(mesh,order=1,dirichlet=dirichlet_bnd,dgjumps=True)
Fhbase = H1(mesh,order=1,dirichlet=dirichlet_bndf,dgjumps=True)
P1 = Compress(Phbase, GetDofsOfElements(Phbase, hasneg))
P2 = Compress(Phbase, GetDofsOfElements(Phbase, haspos))
Pf = Compress(Fhbase, GetDofsOfElements(Fhbase, hasif))
fes = P1*P2*Pf
(p1,p2,pf), (q1,q2,qf) = fes.TnT()

In [8]:
h = specialcf.mesh_size
nf = Normalize(grad(lsetp1)) # normal vector on the fracture
ne = specialcf.normal(2) # normal vector on edges

# Define the jumps and the averages
jump_p1 = p1 - p1.Other()
jump_p2 = p2 - p2.Other()
jump_q1 = q1 - q1.Other()
jump_q2 = q2 - q2.Other()
jump_pf = pf - pf.Other()
jump_qf = qf - qf.Other()

mean_dp1dn = 0.5*Kp*(grad(p1)+grad(p1.Other()))*ne
mean_dq1dn = 0.5*Kp*(grad(q1)+grad(q1.Other()))*ne
mean_dp2dn = 0.5*Kp*(grad(p2)+grad(p2.Other()))*ne
mean_dq2dn = 0.5*Kp*(grad(q2)+grad(q2.Other()))*ne

mean_p1 = 0.5*(p1 + p1.Other())
mean_q1 = 0.5*(q1 + q1.Other())
mean_p2 = 0.5*(p2 + p2.Other())
mean_q2 = 0.5*(q2 + q2.Other())
 

In [9]:
# Interior faces in Omega_neg
interior_neg_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
interior_pos_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=haspos)
gp_neg_faces = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
gp_pos_faces = GetFacetsWithNeighborTypes(mesh, a=haspos, b=hasif)
gp_if_faces = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasif) 

In [10]:
# Element-wise integrals
dx_neg = dCut(lsetp1, NEG, definedonelements=hasneg)
dx_pos = dCut(lsetp1, POS, definedonelements=haspos)
dgamma = dCut(lsetp1, IF, definedonelements=hasif)

# Interior skeleton integrals:
dk_neg = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_neg_facets)
dk_pos = dCut(lsetp1, POS, skeleton=True, definedonelements=interior_pos_facets)

# Domain boundary integrals
# dso_neg = ds(skeleton=True,definedonelements=ci.GetElementsOfType(HASNEG))
# dso_pos = ds(skeleton=True,definedonelements=ci.GetElementsOfType(HASPOS))
dso_neg = ds(skeleton=True, definedon=mesh.Boundaries(dirichlet_bnd))
dso_pos = ds(skeleton=True, definedon=mesh.Boundaries(dirichlet_bnd))

# Ghost penalty integrals
dw_neg = dFacetPatch(definedonelements=gp_neg_faces)
dw_pos = dFacetPatch(definedonelements=gp_pos_faces)
dw_if = dFacetPatch(definedonelements=gp_if_faces)

# f1 = CoefficientFunction(1)
# print(Integrate(f1*dso, mesh))

In [11]:
Ah = BilinearForm(fes)

# A_DG
Ah += Kp*grad(p1)*grad(q1)*dx_neg \
        - (mean_dp1dn*jump_q1 + mean_dq1dn*jump_p1 - beta_p/h*jump_p1*jump_q1)*dk_neg \
        - (Kp*grad(p1)*ne*q1 + Kp*grad(q1)*ne*p1 - beta_p/h*p1*q1)*dso_neg
Ah += Kp*grad(p2)*grad(q2)*dx_pos \
        - (mean_dp2dn*jump_q2 + mean_dq2dn*jump_p2 - beta_p/h*jump_p2*jump_q2)*dk_pos \
        - (Kp*grad(p2)*ne*q2 + Kp*grad(q2)*ne*p2 - beta_p/h*p2*q2)*dso_pos
# Af
Ah += d*Kft*grad(pf)*grad(qf)*dgamma

# I 
Ah += betaf*(p1-p2)*(q1-q2)*dgamma
Ah += alphaf*(0.5*(p1+p2)-pf)*(0.5*(q1+q2)-qf)*dgamma

# ghost penalty for pressure
Ah += sigma_p / (h**2) * (p1 - p1.Other()) * (q1 - q1.Other()) * dw_neg
Ah += sigma_p / (h**2) * (p2 - p2.Other()) * (q2 - q2.Other()) * dw_pos
Ah += sigma_f / (h**2) * (pf - pf.Other()) * (qf - qf.Other()) * dw_if

Ah.Assemble()

In [12]:
# r.h.s
lh = LinearForm(fes)

lh += fp*q1*dx_neg - Kp*grad(q1)*ne*gD*dso_neg + beta_p/h*gD*q1*dso_neg
lh += fp*q2*dx_pos - Kp*grad(q2)*ne*gD*dso_pos + beta_p/h*gD*q2*dso_pos
lh += ff*qf*dgamma

lh.Assemble()

In [13]:
gfu = GridFunction(fes)
freedofs = fes.FreeDofs()
gfu.vec.data += Ah.mat.Inverse(freedofs) * lh.vec

In [14]:
mask_neg = IfPos(levelset,0,1)
mask_pos = IfPos(levelset,1,0)
Draw(mask_neg*gfu.components[0]+mask_pos*gfu.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [15]:
mask_f = IfPos(y-0.5,1,0)
Draw(mask_f*gfu.components[2],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [16]:
vtk = VTKOutput(mesh,[gfu.components[0],gfu.components[1],gfu.components[2]],['p1','p2','q'],"/mnt/d/ngs_output/DFM/singleimmersed",subdivision=0)
vtk.Do() 

'/mnt/d/ngs_output/DFM/singleimmersed'